Recheck derivative and hessian calculations

Recheck loss calculation for training

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import time
from torch.optim.lr_scheduler import StepLR
import sympy as sp

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [3]:
def numerical_derivative(tensor, epsilon=1e-5):
    shape = tensor.shape
    flat_tensor = tensor.view(-1)
    grad = torch.zeros_like(flat_tensor)
    
    for i in range(flat_tensor.numel()):
        original_value = flat_tensor[i].item()
        
        flat_tensor[i] = original_value + epsilon
        perturbed_tensor = flat_tensor.view(shape)
        f_plus = perturbed_tensor
        
        flat_tensor[i] = original_value - epsilon
        perturbed_tensor = flat_tensor.view(shape)
        f_minus = perturbed_tensor
        
        grad[i] = (f_plus - f_minus).mean() / (2 * epsilon)
        flat_tensor[i] = original_value
    
    return grad.view(shape)

In [4]:
def loss_func(self, output, target):
        target_range = torch.max(target, dim=-1, keepdim=True)[0] - torch.min(target, dim=-1, keepdim=True)[0]
        target_range = torch.clamp(target_range, min=1e-6).squeeze(-1)
        loss = ((target - output)**2)/target_range
        l2_reg = sum(p.pow(2.0).sum() for p in self.parameters())
        l1_reg = sum(p.abs().sum() for p in self.parameters())
        total_loss = loss + 0.01*l2_reg + 0.01*l1_reg
        return total_loss

def model_evaluate_function(self, params, symbols, formula):
    var_values = {symbols[j]: params[:, j] for j in range(len(symbols))}
    lambda_func = sp.lambdify(symbols, formula, modules=['numpy'])
    np_values = {str(sym): var_values[sym].detach().cpu().numpy() for sym in symbols}
    with np.errstate(all='ignore'): #need to fix this at some point
        evaluated = lambda_func(**np_values)
    evaluated = np.nan_to_num(evaluated, nan=0.0)
    return torch.tensor(evaluated, dtype=torch.float32, requires_grad=True)

In [5]:
class Multi_Func(nn.Module):
    def __init__(self, functions, num_params, symbols, input_channels, device):
        super().__init__()
        self.device = device
        self.functions = functions
        self.num_params = num_params
        self.symbols = symbols
        self.input_channels = input_channels
        self.params = sum(self.num_params)

        self.hidden_x1 = nn.Sequential(
            nn.Conv1d(in_channels=self.input_channels, out_channels=8, kernel_size=1),
            nn.SELU(),
            nn.Conv1d(in_channels=8, out_channels=6, kernel_size=1),
            nn.SELU(),
            nn.Conv1d(in_channels=6, out_channels=4, kernel_size=1),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(64)
        )

        self.hidden_xfc = nn.Sequential(
            nn.Linear(256, 64),
            nn.SELU(),
            nn.Linear(64, 32),
            nn.SELU(),
            nn.Linear(32, 20),
            nn.SELU(),
        )

        self.hidden_x2 = nn.Sequential(
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=2, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(16),
            nn.Conv1d(in_channels=4, out_channels=2, kernel_size=3),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(8),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=3),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(4),
        )

        self.flatten_layer = nn.Flatten()

        self.hidden_embedding = nn.Sequential(
            nn.Linear(28, 128),
            nn.SELU(),
            nn.Linear(128, 64),
            nn.SELU(),
            nn.Linear(64, self.params),
        )

    def loss_func(self, output, target):
        target_range = torch.max(target, dim=-1, keepdim=True)[0] - torch.min(target, dim=-1, keepdim=True)[0]
        target_range = torch.clamp(target_range, min=1e-6).squeeze(-1)
        #print(f"target_range: {target_range}")
        loss = ((target - output)**2)/target_range
        l2_reg = sum(p.pow(2.0).sum() for p in self.parameters())
        l1_reg = sum(p.abs().sum() for p in self.parameters())
        total_loss = loss + 0.01*l2_reg + 0.01*l1_reg
        return total_loss

    def evaluate_function(self, params, symbols, formula):
        var_values = {symbols[j]: params[:, j] for j in range(len(symbols))}
        lambda_func = sp.lambdify(symbols, formula, modules=['numpy'])
        np_values = {str(sym): var_values[sym].detach().cpu().numpy() for sym in symbols}
        with np.errstate(all='ignore'): #need to fix this at some point
            evaluated = lambda_func(**np_values)
        evaluated = np.nan_to_num(evaluated, nan=0.0)
        return torch.tensor(evaluated, dtype=torch.float32, requires_grad=True)

    def forward(self, x, n=-1):
        x = x.unsqueeze(1).unsqueeze(2)
        out = self.hidden_x1(x)
        xfc = torch.reshape(out, (n, 256))
        xfc = self.hidden_xfc(xfc)

        out = torch.reshape(out, (n, 2, 128))
        out = self.hidden_x2(out)
        cnn_flat = self.flatten_layer(out)
        encoded = torch.cat((cnn_flat, xfc), 1)
        embedding = self.hidden_embedding(encoded)
        
        start_index = 0
        losses = []
        outputs = []
        #print(self.params)
        #print(f"whole embedding: {embedding.shape}")
        
        for f in range(len(self.functions)):
            #print(f"embedding: {embedding[:, start_index:start_index+self.num_params[f]].shape}")
            output = self.evaluate_function(
                embedding[:, start_index:start_index+self.num_params[f]],
                self.symbols[f],
                self.functions[f]
            ).to(device)
            outputs.append(output)
            loss = self.loss_func(output, x[:,0,0])
            losses.append(loss)
            start_index += self.num_params[f]  
        
        #print(f"loss len: {len(losses)}")      
        '''best_index = torch.argmin(torch.tensor(losses))
        best_func = self.functions[best_index]
        best_loss, best_out = losses[best_index], outputs[best_index]'''

        stacked_outputs = torch.stack(outputs)
        stacked_losses = torch.stack(losses)
        #print(f"stacked_outputs shape: {stacked_outputs.shape}")
        #print(f"stacked_losses shape: {stacked_losses.shape}")
        best_loss, best_indexes = torch.min(stacked_losses, dim=0) 
        #print(f"best_loss shape: {best_loss.shape}")
        #print(f"best_indexes shape: {best_indexes.shape}")
        #print(f"best indexes 0: {best_indexes[0]}")
        best_out = stacked_outputs[best_indexes, -1]
        best_func = [self.functions[idx] for idx in best_indexes]
        #print(f"best_func for 0: {best_func[0]}")

        return best_out, best_loss, best_func, outputs, losses

In [6]:
class Encoder(nn.Module):
    def __init__(self, functions, num_params, symbols, input_channels, device):
        super().__init__()
        self.device = device
        self.functions = functions
        self.num_params = num_params
        self.symbols = symbols
        self.input_channels = input_channels
        self.params = sum(self.num_params)
        self.sequence_length=96
        self.input_channel=2
        self.cov1d_size=128

        self.cov1d = nn.Conv1d(self.input_channel, self.cov1d_size, 3, stride=1, padding=1)
        self.flattened_size = self.cov1d_size * self.sequence_length
        self.dense = nn.Linear(self.flattened_size, self.num_params)
        
        self.selu_1 = nn.SELU()

    def forward(self, x):
        print(f"out initial: {x.shape}")
        out = self.cov1d(x)
        print(f"out after cov1d: {out.shape}")
        out = out.reshape(out.size(0), -1)
        print(f"out before dense: {out.shape}")
        out = self.dense(out)
        out = self.selu_1(out)
        
        start_index = 0
        losses = []
        outputs = []
        
        for f in range(len(self.functions)):
            output = model_evaluate_function(
                out[:, start_index:start_index+self.num_params[f]],
                self.symbols[f],
                self.functions[f]
            ).to(device)
            outputs.append(output)
            loss = loss_func(output, x[:,0,0])
            losses.append(loss)
            start_index += self.num_params[f]  
        
        stacked_outputs = torch.stack(outputs)
        stacked_losses = torch.stack(losses)
        best_loss, best_indexes = torch.min(stacked_losses, dim=0) 
        best_out = stacked_outputs[best_indexes, -1]
        best_func = [self.functions[idx] for idx in best_indexes]

        return best_out, best_loss, best_func, outputs, losses

In [113]:
class Multi_Func_ThreeChannels(nn.Module):
    def __init__(self, functions, num_params, symbols, device):
        super().__init__()
        self.device = device
        self.formulas = functions
        self.num_params = num_params
        self.symbols = symbols
        self.params = sum(self.num_params)

        self.hidden_x1 = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=8, kernel_size=1),
            nn.SELU(),
            nn.Conv1d(in_channels=8, out_channels=6, kernel_size=1),
            nn.SELU(),
            nn.Conv1d(in_channels=6, out_channels=4, kernel_size=1),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(64)
        )

        self.hidden_xfc = nn.Sequential(
            nn.Linear(256, 64),
            nn.SELU(),
            nn.Linear(64, 32),
            nn.SELU(),
            nn.Linear(32, 20),
            nn.SELU(),
        )

        self.hidden_x2 = nn.Sequential(
            nn.MaxPool1d(kernel_size=4),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=5),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(16),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=3),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(8),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=3),
            nn.SELU(),
            nn.AdaptiveAvgPool1d(4),
        )

        self.flatten_layer = nn.Flatten()

        self.hidden_embedding = nn.Sequential(
            nn.Linear(28, 128),
            nn.SELU(),
            nn.Linear(128, 64),
            nn.SELU(),
            nn.Linear(64, self.params),
        )

    def loss_func(self, outputs, targets):
        # outputs and targets lists of tensors (one for each channel)
        losses = []
        for output, target in zip(outputs, targets):
            #target_range = torch.max(target, dim=-1, keepdim=True)[0] - torch.min(target, dim=-1, keepdim=True)[0]
            #target_range = torch.clamp(target_range, min=1e-6).squeeze(-1)
            #print(target.shape, output.shape, target_range.shape)
            loss = (torch.abs(target - output)**2)# / target_range
            losses.append(loss.mean(dim=1))

        total_losses = torch.stack(losses).mean(dim=1)
        l2_reg = sum(p.pow(2.0).sum() for p in self.parameters())
        l1_reg = sum(p.abs().sum() for p in self.parameters())
        total_losses += 0.01 * l2_reg + 0.01 * l1_reg
        return total_losses

    def evaluate_function(self, params, symbols, formula):
        var_values = {symbols[j]: params[:, j] for j in range(len(symbols))}
        lambda_func = sp.lambdify(symbols, formula, modules=['numpy'])
        np_values = {str(sym): var_values[sym].detach().cpu().numpy() for sym in symbols}
        with np.errstate(all='ignore'): #need to fix this at some point
            evaluated = lambda_func(**np_values)
        evaluated = np.nan_to_num(evaluated, nan=0.0)
        return torch.tensor(evaluated, dtype=torch.float32, requires_grad=True)

    def compute_derivative(self, params, index, epsilon=1e-6):
        batch_size = params.shape[0]
        num_params = params.shape[1]
        gradients = torch.zeros(batch_size, num_params)
        param_tensor = params.clone().detach().requires_grad_(True)

        for j in range(num_params):
            perturbed_params_pos = param_tensor.clone()
            perturbed_params_neg = param_tensor.clone()
            perturbed_params_pos[:, j] += epsilon
            forward_value = self.evaluate_function(perturbed_params_pos, self.symbols[index], self.formulas[index])
            perturbed_params_neg[:, j] -= epsilon
            backward_value = self.evaluate_function(perturbed_params_neg, self.symbols[index], self.formulas[index])
            gradients[:, j] = (forward_value - backward_value) / (2 * epsilon)
        return gradients

    def compute_hessians(self, params, index, epsilon=1e-6):
        batch_size = params.shape[0]
        num_params = params.shape[1]
        hessians = torch.zeros((batch_size, num_params, num_params))
        param_tensor = params.clone().detach().requires_grad_(True)
        print(param_tensor.shape)
        for j in range(num_params):
                for k in range(num_params):
                    perturbed_params = param_tensor.clone()
                    perturbed_params[:,j] += epsilon
                    perturbed_params[:,k] += epsilon
                    f_plus_plus = self.evaluate_function(perturbed_params, self.symbols[index], self.formulas[index])                    
                    perturbed_params[:,k] -= 2 * epsilon
                    f_plus_minus = self.evaluate_function(perturbed_params, self.symbols[index], self.formulas[index])                   
                    perturbed_params[:,j] -= 2 * epsilon
                    perturbed_params[:,k] += 2 * epsilon
                    f_minus_plus = self.evaluate_function(perturbed_params, self.symbols[index], self.formulas[index])                    
                    perturbed_params[:,k] -= 2 * epsilon
                    f_minus_minus = self.evaluate_function(perturbed_params, self.symbols[index], self.formulas[index])
                    hessians[:, j, k] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * epsilon**2)
        return hessians

    def forward(self, x):
        out = self.hidden_x1(x.swapaxes(1, 2))
        xfc = torch.reshape(out, (-1, 256))
        xfc = self.hidden_xfc(xfc)

        out = torch.reshape(out, (-1, 2, 128))
        out = self.hidden_x2(out)
        cnn_flat = self.flatten_layer(out)
        encoded = torch.cat((cnn_flat, xfc), 1)
        embedding = self.hidden_embedding(encoded)
        
        start_index = 0
        losses = []
        outputs = []
        preds = []

        for f in range(len(self.formulas)):
            output = self.evaluate_function(
                embedding[:, start_index:start_index+self.num_params[f]],
                self.symbols[f],
                self.formulas[f]
            ).to(self.device)
            outputs.append(output)
            der = self.compute_derivative(embedding[:, start_index:start_index+self.num_params[f]], f).to(self.device)
            hess = self.compute_hessians(embedding[:, start_index:start_index+self.num_params[f]], f).to(self.device)
            
            hess = torch.flatten(hess, start_dim=1)
            output = output.unsqueeze(1)
            output = F.pad(output, (0, hess.size(1) - output.size(1)))
            der = F.pad(der, (0, hess.size(-1) - der.size(-1)))

            pred = torch.stack([output, der, hess], dim=2)
            print(f"pred shape {f}: {pred.shape}")
            print(f"pred {f}: {pred}")
            pred = F.pad(pred, (0, 0, (x.size(1) - pred.size(1)), 0, 0, 0))
            #print(f"pred {f}: {pred.shape}")
            #print(f"x {f}: {x.shape}")
            preds.append(pred)
            loss = self.loss_func(pred, x)
            losses.append(loss)
            start_index += self.num_params[f]  

        stacked_outputs = torch.stack(outputs)
        stacked_losses = torch.stack(losses)
        stacked_preds = torch.stack(preds)
        best_loss, best_indexes = torch.min(stacked_losses, dim=0)
        best_out = stacked_outputs[best_indexes, -1]
        best_func = [self.formulas[idx] for idx in best_indexes]

        return best_out, best_loss, best_func, stacked_outputs, stacked_losses, stacked_preds


In [5]:
loaded_data = torch.load('hold_data.pth')

results = loaded_data['results']
formulas = loaded_data['formulas']
symbols = loaded_data['symbols']
params = loaded_data['params']
num_params = loaded_data['num_params']
derivatives = loaded_data['derivatives']
hessians = loaded_data['hessians']
full_data = loaded_data['full_data']
flat_data = loaded_data['flattened_data']

dataloader = DataLoader(flat_data, batch_size=50, shuffle=True)

/tmp/ipykernel_1945724/2816734550.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load('hold_data.pth')


In [16]:
derivatives.shape, hessians.shape

(torch.Size([10, 1000, 81]), torch.Size([10, 1000, 81]))

In [16]:
derivatives[0, 0]

tensor([-0.1030,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000])

In [14]:
flat_data[0, :, 1]

tensor([-0.1030,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000])

In [22]:
params[0][0]

tensor([[[2.0002]]], device='cuda:4')

In [23]:
results[0][0]

tensor(0.0540)

In [25]:
formulas[0]

sqrt(2)*exp(-theta**2/2)/(2*sqrt(pi))

In [18]:
def evaluate_function(params, symbols, formula):
    if params.shape[0] != 1:
        var_values = {symbols[j]: params[j].item() for j in range(len(symbols))}
    else:
        var_values = {symbols[j]: params[:, j, :].item() for j in range(len(symbols))}
    evaluated = formula.subs(var_values)
    return float(evaluated)

In [6]:
def training_loss_func(model, output, target):
    target_max = torch.max(target, dim=-1, keepdim=True)[0]
    target_min = torch.min(target, dim=-1, keepdim=True)[0]
    target_range = torch.clamp(target_max - target_min, min=1e-6).squeeze(-1)
    
    mse_loss = torch.mean((output - target) ** 2, dim=-1)
    normalized_loss = mse_loss / target_range
    
    l2_reg = sum(p.pow(2.0).sum() for p in model.parameters())
    l1_reg = sum(p.abs().sum() for p in model.parameters())
    
    output_derivative = numerical_derivative(output)
    target_derivative = numerical_derivative(target)
    
    derivative_diff = torch.mean((output_derivative - target_derivative) ** 2, dim=-1)
    
    total_loss = torch.mean(normalized_loss) + 0.01 * l2_reg + 0.01 * l1_reg + 0.1 * torch.mean(derivative_diff)
    return total_loss


In [114]:
model = Multi_Func_ThreeChannels(functions=formulas, num_params=num_params, symbols=symbols, device=device).to(device)

#train_loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

epochs = 10
for epoch in range(epochs):
    start_time = time.time()
    train_loss = 0.0
    total_num = 1
    model.train()
    
    for train_batch in dataloader:
        train_batch = train_batch.to(device)
        optimizer.zero_grad()
        best_out, best_loss, best_func, outputs, losses, preds = model(train_batch)
        loss = training_loss_func(model, best_out, train_batch[:, 0, 0]) #train_loss_func(best_out, train_batch[:,0,0])
        '''print(f"target out: {train_batch[0, 0, 0]}")
        print(f"best_out: {best_out[0]}")
        print(f"target der: {train_batch[0, :, 1]}")
        print(f"pred ders: {preds[:, 0, :, 1]}")
        print(f"preds: {preds.shape}")
        print(f"target: {train_batch.shape}\n")'''
        break;
        ''' print(f"best_loss: {best_loss[0]}")
        print(f"all losses: {losses[:, 0]}")'''
        print(f"all outputs: {outputs[:, 0]}")
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * best_out.shape[0]
        total_num += best_out.shape[0]
    '''scheduler.step()
    train_loss /= total_num
    print(f"epoch : {epoch}/{epochs}, loss = {train_loss:.8f}")
    print(f"--- {time.time() - start_time} seconds ---")'''

torch.Size([50, 1])
output 0: torch.Size([50, 1])
der 0: torch.Size([50, 1])
hess 0: torch.Size([50, 1])
output 0: tensor([[0.3982],
        [0.3982],
        [0.3983],
        [0.3983],
        [0.3983],
        [0.3982],
        [0.3982],
        [0.3986],
        [0.3982],
        [0.3982],
        [0.3983],
        [0.3983],
        [0.3982],
        [0.3982],
        [0.3983],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3984],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3984],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3983],
        [0.3982],
        [0.3982],
        [0.3984],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3983],
        [0.3982],
        [0.3983],
        [0.3982],
        [0.3985],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3982],
        [0.3983],
        [0.3983],
        [0.3987],
        [0.3983],
   

In [3]:
'''r = np.random.randint(data.shape[0])
print(data[r].shape)
plt.plot(data[r].detach().numpy());'''

'r = np.random.randint(data.shape[0])\nprint(data[r].shape)\nplt.plot(data[r].detach().numpy());'